In [ ]:
'''
Input: Mask video, target logo
Processing: 
After having gotten all the details of fps, height, width, etc:
a) Define the color ranges to look for
b) Identify the contours frame by frame and overlay them after resizing the target logo appropriately
Output: 
'''

import cv2
import numpy as np

# Path to your video and the PES University logo
video_path = '/content/maskedvideo.mp4'  # Update based on your Google Drive path
logo_path = '/content/PESLogo.jpg'  # This is the PES University logo image you uploaded

# Open the video file
cap = cv2.VideoCapture(video_path)

# Load the PES University logo
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_with_rotated_logos.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Define color ranges for orange (#d8a94d) and teal (#32bfaf) in HSV format
orange_lower = np.array([17, 50, 50])  # Looser range around orange hue
orange_upper = np.array([19, 255, 255])

teal_lower = np.array([85, 50, 50])  # Looser range around teal hue
teal_upper = np.array([90, 255, 255])

# Function to detect and mask the sponsor spots based on color ranges
def detect_sponsor_spots(frame, lower_color, upper_color):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_frame, lower_color, upper_color)
    return mask

# Function to overlay the logo within a rotated bounding box
def overlay_rotated_logo(frame, logo, box_points):
    # Order the box points correctly
    box_points = np.int0(box_points)

    # Extract the region of interest (ROI) from the frame
    width = int(np.linalg.norm(box_points[0] - box_points[1]))
    height = int(np.linalg.norm(box_points[1] - box_points[2]))

    # Check if width or height is zero or negative
    if width <= 0 or height <= 0:
        return  # Skip resizing and overlaying if dimensions are invalid

    # Get the rotation matrix for the bounding box
    src_pts = np.float32([box_points[0], box_points[1], box_points[2]])
    dst_pts = np.float32([[0, 0], [width - 1, 0], [0, height - 1]])
    M = cv2.getAffineTransform(src_pts, dst_pts)

    # Warp the region of the frame to get the upright ROI
    warped_frame = cv2.warpAffine(frame, M, (width, height))

    # Resize the logo to fit the bounding box dimensions
    resized_logo = cv2.resize(logo, (width, height))

    # If the logo has an alpha channel, blend the logo with transparency
    if logo.shape[2] == 4:
        alpha_logo = resized_logo[:, :, 3] / 255.0
        alpha_frame = 1.0 - alpha_logo

        for c in range(0, 3):
            warped_frame[:, :, c] = (alpha_logo * resized_logo[:, :, c] +
                                     alpha_frame * warped_frame[:, :, c])
    else:
        warped_frame[:, :, 0:3] = resized_logo

    # Reverse the warp to put the transformed region back in the original frame
    M_inv = cv2.getAffineTransform(dst_pts, src_pts)
    transformed_frame = cv2.warpAffine(warped_frame, M_inv, (frame.shape[1], frame.shape[0]))

    # Overlay the transformed logo onto the original frame
    frame = cv2.addWeighted(frame, 1, transformed_frame, 1, 0)

# Process each frame in the video
frame_count = 0  # Track frame count
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    print(f"Processing frame {frame_count}...")

    # Process orange mask separately
    orange_mask = detect_sponsor_spots(frame, orange_lower, orange_upper)
    contours, _ = cv2.findContours(orange_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Get the rotated bounding box for the contour
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        overlay_rotated_logo(frame, logo_image, box)

    # Process teal mask separately
    teal_mask = detect_sponsor_spots(frame, teal_lower, teal_upper)
    contours, _ = cv2.findContours(teal_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Get the rotated bounding box for the contour
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        overlay_rotated_logo(frame, logo_image, box)

    # Write the modified frame with logos to the output video
    out.write(frame)

# Release the video objects
cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete. Total frames processed:", frame_count)


Processing frame 1...


<ipython-input-16-9612843e72da>:39: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)


Processing frame 2...
Processing frame 3...
Processing frame 4...
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Processing frame 10...
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Processing frame 20...
Processing frame 21...
Processing frame 22...
Processing frame 23...
Processing frame 24...
Processing frame 25...
Processing frame 26...
Processing frame 27...
Processing frame 28...
Processing frame 29...
Processing frame 30...
Processing frame 31...
Processing frame 32...
Processing frame 33...
Processing frame 34...
Processing frame 35...
Processing frame 36...
Processing frame 37...
Processing frame 38...
Processing frame 39...
Processing frame 40...
Processing frame 41...
Processing frame 42...
Processing frame 43...
Processing frame 44...
Processing frame 45

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete. Total frames processed: 240


In [ ]:
def overlay_rotated_logo(frame, logo, box_points):
    # Order the box points correctly
    box_points = np.int0(box_points)

    # Extract the region of interest (ROI) from the frame
    width = int(np.linalg.norm(box_points[0] - box_points[1]))
    height = int(np.linalg.norm(box_points[1] - box_points[2]))

    # Check if width or height is zero or negative
    if width <= 0 or height <= 0:
        return  # Skip resizing and overlaying if dimensions are invalid

    # Get the rotation matrix for the bounding box
    src_pts = np.float32([box_points[0], box_points[1], box_points[2]])
    dst_pts = np.float32([[0, 0], [width - 1, 0], [0, height - 1]])
    M = cv2.getAffineTransform(src_pts, dst_pts)

    # Warp the region of the frame to get the upright ROI
    warped_frame = cv2.warpAffine(frame, M, (width, height))

    # Resize the logo to fit the bounding box dimensions
    resized_logo = cv2.resize(logo, (width, height))

    # If the logo has an alpha channel, blend the logo with transparency
    if logo.shape[2] == 4:
        alpha_logo = resized_logo[:, :, 3] / 255.0
        alpha_frame = 1.0 - alpha_logo

        for c in range(0, 3):
            warped_frame[:, :, c] = (alpha_logo * resized_logo[:, :, c] +
                                     alpha_frame * warped_frame[:, :, c])
    else:
        warped_frame[:, :, 0:3] = resized_logo

    # Reverse the warp to put the transformed region back in the original frame
    M_inv = cv2.getAffineTransform(dst_pts, src_pts)
    transformed_frame = cv2.warpAffine(warped_frame, M_inv, (frame.shape[1], frame.shape[0]))

    # Overlay the transformed logo onto the original frame
    frame = cv2.addWeighted(frame, 1, transformed_frame, 1, 0)

In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get video properties (assuming both videos have the same properties)
frame_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(original_cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_with_contours.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Function to detect and draw contours on the original video based on the mask
def draw_contours_on_frame(frame, mask):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw all contours on the frame
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)  # Draw in green with thickness 2

    # Debug: print how many contours were detected
    print(f"Detected {len(contours)} contours.")

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()
    ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Draw the detected contours on the original frame
    draw_contours_on_frame(original_frame, binary_mask)

    # Write the modified frame to the output video
    out.write(original_frame)

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


Detected 35 contours.
Detected 37 contours.
Detected 29 contours.
Detected 38 contours.
Detected 36 contours.
Detected 39 contours.
Detected 42 contours.
Detected 39 contours.
Detected 40 contours.
Detected 44 contours.
Detected 38 contours.
Detected 37 contours.
Detected 37 contours.
Detected 38 contours.
Detected 34 contours.
Detected 38 contours.
Detected 36 contours.
Detected 34 contours.
Detected 35 contours.
Detected 36 contours.
Detected 32 contours.
Detected 33 contours.
Detected 35 contours.
Detected 35 contours.
Detected 34 contours.
Detected 36 contours.
Detected 31 contours.
Detected 33 contours.
Detected 33 contours.
Detected 32 contours.
Detected 32 contours.
Detected 35 contours.
Detected 32 contours.
Detected 32 contours.
Detected 36 contours.
Detected 36 contours.
Detected 33 contours.
Detected 33 contours.
Detected 32 contours.
Detected 32 contours.
Detected 35 contours.
Detected 36 contours.
Detected 32 contours.
Detected 38 contours.
Detected 36 contours.
Detected 3

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get video properties for the original video
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Get video properties for the mask video
mask_width = int(mask_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
mask_height = int(mask_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Print the resolutions
print(f"Original Video Resolution: {original_width}x{original_height}")
print(f"Mask Video Resolution: {mask_width}x{mask_height}")

# Release the video objects
original_cap.release()
mask_cap.release()


Original Video Resolution: 1920x1080
Mask Video Resolution: 1280x720


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(original_cap.get(cv2.CAP_PROP_FPS))

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_filtered_contours.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (target_width, target_height))

# Function to detect and draw filtered contours on the original video based on the mask
def draw_filtered_contours_on_frame(frame, mask, width, height, border_threshold=10):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out contours that are too close to the border (less than 10 pixels away)
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x > border_threshold and y > border_threshold and (x + w) < (width - border_threshold) and (y + h) < (height - border_threshold):
            filtered_contours.append(contour)

    # Draw the filtered contours on the frame
    cv2.drawContours(frame, filtered_contours, -1, (0, 255, 0), 2)  # Draw in green with thickness 2

    # Debug: print how many contours were filtered and kept
    print(f"Filtered {len(filtered_contours)} contours out of {len(contours)} detected.")

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()
    ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Draw the filtered contours on the rescaled frame
    draw_filtered_contours_on_frame(rescaled_frame, rescaled_mask, target_width, target_height)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


Filtered 14 contours out of 35 detected.
Filtered 16 contours out of 37 detected.
Filtered 8 contours out of 29 detected.
Filtered 17 contours out of 38 detected.
Filtered 15 contours out of 36 detected.
Filtered 18 contours out of 39 detected.
Filtered 21 contours out of 42 detected.
Filtered 18 contours out of 39 detected.
Filtered 19 contours out of 40 detected.
Filtered 23 contours out of 44 detected.
Filtered 17 contours out of 38 detected.
Filtered 16 contours out of 37 detected.
Filtered 16 contours out of 37 detected.
Filtered 17 contours out of 38 detected.
Filtered 13 contours out of 34 detected.
Filtered 17 contours out of 38 detected.
Filtered 15 contours out of 36 detected.
Filtered 13 contours out of 34 detected.
Filtered 14 contours out of 35 detected.
Filtered 15 contours out of 36 detected.
Filtered 11 contours out of 32 detected.
Filtered 12 contours out of 33 detected.
Filtered 14 contours out of 35 detected.
Filtered 14 contours out of 35 detected.
Filtered 13 conto

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get FPS for the original video
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

# Get FPS for the mask video
mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Print the FPS values
print(f"Original Video FPS: {original_fps}")
print(f"Mask Video FPS: {mask_fps}")

# Release the video objects
original_cap.release()
mask_cap.release()

Original Video FPS: 30.0
Mask Video FPS: 24.0


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to detect and draw filtered contours on the original video based on the mask
def draw_filtered_contours_on_frame(frame, mask, width, height, border_threshold=10):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out contours that are too close to the border (less than 10 pixels away)
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x > border_threshold and y > border_threshold and (x + w) < (width - border_threshold) and (y + h) < (height - border_threshold):
            filtered_contours.append(contour)

    # Draw the filtered contours on the frame
    cv2.drawContours(frame, filtered_contours, -1, (0, 255, 0), 2)  # Draw in green with thickness 2

    # Debug: print how many contours were filtered and kept
    print(f"Filtered {len(filtered_contours)} contours out of {len(contours)} detected.")

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Draw the filtered contours on the rescaled frame
    draw_filtered_contours_on_frame(rescaled_frame, rescaled_mask, target_width, target_height)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

Filtered 14 contours out of 35 detected.
Filtered 14 contours out of 35 detected.
Filtered 16 contours out of 37 detected.
Filtered 8 contours out of 29 detected.
Filtered 17 contours out of 38 detected.
Filtered 15 contours out of 36 detected.
Filtered 15 contours out of 36 detected.
Filtered 18 contours out of 39 detected.
Filtered 21 contours out of 42 detected.
Filtered 18 contours out of 39 detected.
Filtered 19 contours out of 40 detected.
Filtered 19 contours out of 40 detected.
Filtered 23 contours out of 44 detected.
Filtered 17 contours out of 38 detected.
Filtered 16 contours out of 37 detected.
Filtered 16 contours out of 37 detected.
Filtered 16 contours out of 37 detected.
Filtered 17 contours out of 38 detected.
Filtered 13 contours out of 34 detected.
Filtered 17 contours out of 38 detected.
Filtered 15 contours out of 36 detected.
Filtered 15 contours out of 36 detected.
Filtered 13 contours out of 34 detected.
Filtered 14 contours out of 35 detected.
Filtered 15 conto

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to detect and draw bounding boxes around contours on the original video based on the mask
def draw_bounding_box_on_frame(frame, mask, width, height, border_threshold=10):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out contours that are too close to the border (less than 10 pixels away)
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x > border_threshold and y > border_threshold and (x + w) < (width - border_threshold) and (y + h) < (height - border_threshold):
            filtered_contours.append((x, y, w, h))

    # Draw bounding boxes around the filtered contours on the frame
    for (x, y, w, h) in filtered_contours:
        # Draw a rectangle (bounding box) on the frame
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding box with thickness 2

    # Debug: print how many bounding boxes were drawn
    print(f"Bounding boxes drawn: {len(filtered_contours)}")

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Draw bounding boxes on the rescaled frame
    draw_bounding_box_on_frame(rescaled_frame, rescaled_mask, target_width, target_height)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

Bounding boxes drawn: 14
Bounding boxes drawn: 14
Bounding boxes drawn: 16
Bounding boxes drawn: 8
Bounding boxes drawn: 17
Bounding boxes drawn: 15
Bounding boxes drawn: 15
Bounding boxes drawn: 18
Bounding boxes drawn: 21
Bounding boxes drawn: 18
Bounding boxes drawn: 19
Bounding boxes drawn: 19
Bounding boxes drawn: 23
Bounding boxes drawn: 17
Bounding boxes drawn: 16
Bounding boxes drawn: 16
Bounding boxes drawn: 16
Bounding boxes drawn: 17
Bounding boxes drawn: 13
Bounding boxes drawn: 17
Bounding boxes drawn: 15
Bounding boxes drawn: 15
Bounding boxes drawn: 13
Bounding boxes drawn: 14
Bounding boxes drawn: 15
Bounding boxes drawn: 11
Bounding boxes drawn: 11
Bounding boxes drawn: 12
Bounding boxes drawn: 14
Bounding boxes drawn: 14
Bounding boxes drawn: 13
Bounding boxes drawn: 13
Bounding boxes drawn: 15
Bounding boxes drawn: 10
Bounding boxes drawn: 12
Bounding boxes drawn: 12
Bounding boxes drawn: 12
Bounding boxes drawn: 11
Bounding boxes drawn: 11
Bounding boxes drawn: 14
B

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to detect and draw rotated bounding boxes on the original video based on the mask
def draw_rotated_bounding_box_on_frame(frame, mask, width, height, border_threshold=10):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out contours that are too close to the border (less than 10 pixels away)
    filtered_contours = []
    for contour in contours:
        # Ignore small contours or contours close to the border
        rect = cv2.minAreaRect(contour)  # Get the rotated rectangle that bounds the contour
        box = cv2.boxPoints(rect)  # Get the four vertices of the rectangle
        box = np.int0(box)  # Convert the vertices to integers
        x, y, w, h = cv2.boundingRect(box)

        if x > border_threshold and y > border_threshold and (x + w) < (width - border_threshold) and (y + h) < (height - border_threshold):
            filtered_contours.append(box)

    # Draw rotated bounding boxes around the filtered contours on the frame
    for box in filtered_contours:
        # Draw the rotated bounding box on the frame
        cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)  # Draw the box in green with thickness 2

    # Debug: print how many rotated bounding boxes were drawn
    print(f"Rotated bounding boxes drawn: {len(filtered_contours)}")

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Draw rotated bounding boxes on the rescaled frame
    draw_rotated_bounding_box_on_frame(rescaled_frame, rescaled_mask, target_width, target_height)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<ipython-input-40-aba7cdef3d62>:41: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Convert the vertices to integers


Rotated bounding boxes drawn: 14
Rotated bounding boxes drawn: 14
Rotated bounding boxes drawn: 16
Rotated bounding boxes drawn: 8
Rotated bounding boxes drawn: 17
Rotated bounding boxes drawn: 15
Rotated bounding boxes drawn: 15
Rotated bounding boxes drawn: 18
Rotated bounding boxes drawn: 21
Rotated bounding boxes drawn: 18
Rotated bounding boxes drawn: 19
Rotated bounding boxes drawn: 19
Rotated bounding boxes drawn: 23
Rotated bounding boxes drawn: 17
Rotated bounding boxes drawn: 16
Rotated bounding boxes drawn: 16
Rotated bounding boxes drawn: 16
Rotated bounding boxes drawn: 17
Rotated bounding boxes drawn: 13
Rotated bounding boxes drawn: 17
Rotated bounding boxes drawn: 15
Rotated bounding boxes drawn: 15
Rotated bounding boxes drawn: 13
Rotated bounding boxes drawn: 14
Rotated bounding boxes drawn: 15
Rotated bounding boxes drawn: 11
Rotated bounding boxes drawn: 11
Rotated bounding boxes drawn: 12
Rotated bounding boxes drawn: 14
Rotated bounding boxes drawn: 14
Rotated bou

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to detect and draw skewed quadrilaterals on the original video based on the mask
def draw_skewed_quadrilateral_on_frame(frame, mask, width, height, border_threshold=10):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out contours that are too close to the border (less than 10 pixels away)
    filtered_contours = []
    for contour in contours:
        # Approximate the contour to a polygon (quadrilateral)
        epsilon = 0.05 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) == 4:  # If it approximates to a quadrilateral
            x, y, w, h = cv2.boundingRect(approx)
            if x > border_threshold and y > border_threshold and (x + w) < (width - border_threshold) and (y + h) < (height - border_threshold):
                filtered_contours.append(approx)

    # Draw skewed quadrilaterals around the filtered contours on the frame
    for quad in filtered_contours:
        # Draw the quadrilateral on the frame
        cv2.drawContours(frame, [quad], 0, (0, 255, 0), 2)  # Green quadrilateral with thickness 2

    # Debug: print how many quadrilaterals were drawn
    print(f"Skewed quadrilaterals drawn: {len(filtered_contours)}")

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Draw skewed quadrilaterals on the rescaled frame
    draw_skewed_quadrilateral_on_frame(rescaled_frame, rescaled_mask, target_width, target_height)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

Skewed quadrilaterals drawn: 4
Skewed quadrilaterals drawn: 4
Skewed quadrilaterals drawn: 4
Skewed quadrilaterals drawn: 1
Skewed quadrilaterals drawn: 1
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 4
Skewed quadrilaterals drawn: 1
Skewed quadrilaterals drawn: 4
Skewed quadrilaterals drawn: 4
Skewed quadrilaterals drawn: 5
Skewed quadrilaterals drawn: 3
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 3
Skewed quadrilaterals drawn: 3
Skewed quadrilaterals drawn: 1
Skewed quadrilaterals drawn: 3
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 3
Skewed quadrilaterals drawn: 3
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 3
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed quadrilaterals drawn: 2
Skewed q

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video showing where the sponsor should go

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to apply PCA for determining the orientation of the region
def get_pca_orientation(points):
    # Perform PCA on the set of points
    mean, eigenvectors = cv2.PCACompute(points, mean=None)

    # The first eigenvector represents the principal direction (orientation)
    principal_axis = eigenvectors[0]  # Principal axis

    # We return the mean (centroid) and the principal axis
    return mean[0], principal_axis

# Function to draw rotated bounding box based on the curvature and principal axis
def draw_oriented_box_on_curvature(frame, mask, width, height, border_threshold=10):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if len(contour) >= 5:  # At least 5 points to compute PCA meaningfully
            # Reshape the contour points for PCA
            contour_points = contour.reshape(-1, 2).astype(np.float32)

            # Apply PCA to find the principal orientation of the contour
            centroid, principal_axis = get_pca_orientation(contour_points)

            # Fit a rotated bounding box based on the principal axis
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)  # Get the box points of the rotated rectangle
            box = np.int0(box)  # Convert to integer for drawing

            # Draw the oriented bounding box
            cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)  # Green box

            # Optionally, draw the principal axis for debugging
            axis_start = (int(centroid[0]), int(centroid[1]))
            axis_end = (int(centroid[0] + principal_axis[0] * 50), int(centroid[1] + principal_axis[1] * 50))
            cv2.line(frame, axis_start, axis_end, (255, 0, 0), 2)  # Draw principal axis in blue

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Draw oriented bounding box based on curvature
    draw_oriented_box_on_curvature(rescaled_frame, rescaled_mask, target_width, target_height)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<ipython-input-47-45b3717c33dd>:57: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Convert to integer for drawing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/PESLogo.jpg'  # PES University logo image

# Load the PES University logo
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to apply PCA for determining the orientation of the region
def get_pca_orientation(points):
    # Perform PCA on the set of points
    mean, eigenvectors = cv2.PCACompute(points, mean=None)

    # The first eigenvector represents the principal direction (orientation)
    principal_axis = eigenvectors[0]  # Principal axis

    # We return the mean (centroid) and the principal axis
    return mean[0], principal_axis

# Function to warp and overlay the PES logo based on the orientation of the largest region
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Reshape the contour points for PCA
    contour_points = largest_contour.reshape(-1, 2).astype(np.float32)

    # Apply PCA to find the principal orientation of the contour
    centroid, principal_axis = get_pca_orientation(contour_points)

    # Fit a rotated bounding box based on the principal axis
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)  # Get the box points of the rotated rectangle
    box = np.int0(box)  # Convert to integer for drawing

    # Get the size of the PES logo
    logo_h, logo_w = logo.shape[:2]

    # Define the source points (corners of the PES logo)
    src_pts = np.float32([[0, 0], [logo_w, 0], [logo_w, logo_h], [0, logo_h]])

    # Define the destination points (corners of the rotated bounding box)
    dst_pts = np.float32(box)

    # Compute the homography matrix
    homography_matrix, _ = cv2.findHomography(src_pts, dst_pts)

    # Warp the logo to fit the detected region using homography
    warped_logo = cv2.warpPerspective(logo, homography_matrix, (width, height))

    # Extract the alpha channel from the warped logo (for blending if needed)
    if warped_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = warped_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        warped_logo = warped_logo[:, :, :3]  # Remove the alpha channel from the logo image
    else:
        logo_alpha = np.ones((warped_logo.shape[0], warped_logo.shape[1]), dtype=np.float32)

    # Create a mask for the logo region (where the warped logo should be applied)
    logo_mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillConvexPoly(logo_mask, box, 255)  # Mask for the logo region

    # Extract the region of interest from the frame where the logo will be placed
    roi = frame.copy()

    # Blend the warped logo with the original frame using the alpha channel
    for c in range(0, 3):
        roi[:, :, c] = roi[:, :, c] * (1 - logo_alpha) + warped_logo[:, :, c] * logo_alpha

    # Overlay the blended region onto the original frame
    frame[logo_mask > 0] = roi[logo_mask > 0]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the PES logo based on the largest detected region
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<ipython-input-51-2ec7733df708>:69: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Convert to integer for drawing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/target.png'  # PES University logo image

# Load the PES University logo
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to apply PCA for determining the orientation of the region
def get_pca_orientation(points):
    # Perform PCA on the set of points
    mean, eigenvectors = cv2.PCACompute(points, mean=None)

    # The first eigenvector represents the principal direction (orientation)
    principal_axis = eigenvectors[0]  # Principal axis

    # We return the mean (centroid) and the principal axis
    return mean[0], principal_axis

# Initialize smoothing variables for PCA orientation
prev_principal_axis = None
smoothing_factor = 0.1  # Adjust this to control how much smoothing is applied

# Function to smooth the PCA values using exponential moving average (EMA)
def smooth_pca_orientation(current_axis):
    global prev_principal_axis

    if prev_principal_axis is None:
        prev_principal_axis = current_axis
        return current_axis
    else:
        # Smooth the current PCA direction with the previous one
        smoothed_axis = (smoothing_factor * current_axis) + ((1 - smoothing_factor) * prev_principal_axis)
        prev_principal_axis = smoothed_axis
        return smoothed_axis

# Function to warp and overlay the PES logo based on the orientation of the largest region
def warp_and_overlay_logo(frame, mask, width, height, logo, transparency=0.5):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Reshape the contour points for PCA
    contour_points = largest_contour.reshape(-1, 2).astype(np.float32)

    # Apply PCA to find the principal orientation of the contour
    centroid, principal_axis = get_pca_orientation(contour_points)

    # Smooth the PCA principal axis
    principal_axis = smooth_pca_orientation(principal_axis)

    # Fit a rotated bounding box based on the smoothed principal axis
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)  # Get the box points of the rotated rectangle
    box = np.int0(box)  # Convert to integer for drawing

    # Get the size of the PES logo
    logo_h, logo_w = logo.shape[:2]

    # Define the source points (corners of the PES logo)
    src_pts = np.float32([[0, 0], [logo_w, 0], [logo_w, logo_h], [0, logo_h]])

    # Define the destination points (corners of the rotated bounding box)
    dst_pts = np.float32(box)

    # Compute the homography matrix
    homography_matrix, _ = cv2.findHomography(src_pts, dst_pts)

    # Warp the logo to fit the detected region using homography
    warped_logo = cv2.warpPerspective(logo, homography_matrix, (width, height))

    # Extract the alpha channel from the warped logo (for blending if needed)
    if warped_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = warped_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        warped_logo = warped_logo[:, :, :3]  # Remove the alpha channel from the logo image
    else:
        logo_alpha = np.ones((warped_logo.shape[0], warped_logo.shape[1]), dtype=np.float32)

    # Adjust the logo's alpha based on the transparency factor
    logo_alpha *= transparency

    # Create a mask for the logo region (where the warped logo should be applied)
    logo_mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillConvexPoly(logo_mask, box, 255)  # Mask for the logo region

    # Extract the region of interest from the frame where the logo will be placed
    roi = frame.copy()

    # Blend the warped logo with the original frame using the alpha channel and transparency
    for c in range(0, 3):
        roi[:, :, c] = roi[:, :, c] * (1 - logo_alpha) + warped_logo[:, :, c] * logo_alpha

    # Overlay the blended region onto the original frame
    frame[logo_mask > 0] = roi[logo_mask > 0]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the PES logo based on the largest detected region with transparency and smoothing
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image, transparency=0.5)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<ipython-input-52-b94c7c4f3a6f>:89: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Convert to integer for drawing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to apply PCA for determining the orientation of the region
def get_pca_orientation(points):
    # Perform PCA on the set of points
    mean, eigenvectors = cv2.PCACompute(points, mean=None)

    # The first eigenvector represents the principal direction (orientation)
    principal_axis = eigenvectors[0]  # Principal axis

    # We return the mean (centroid) and the principal axis
    return mean[0], principal_axis

# Initialize smoothing variables for PCA orientation
prev_principal_axis = None
smoothing_factor = 0.1  # Adjust this to control how much smoothing is applied

# Function to smooth the PCA values using exponential moving average (EMA)
def smooth_pca_orientation(current_axis):
    global prev_principal_axis

    if prev_principal_axis is None:
        prev_principal_axis = current_axis
        return current_axis
    else:
        # Smooth the current PCA direction with the previous one
        smoothed_axis = (smoothing_factor * current_axis) + ((1 - smoothing_factor) * prev_principal_axis)
        prev_principal_axis = smoothed_axis
        return smoothed_axis

# Function to fill the bounding box with the color of its boundary
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (average color of corners)
    boundary_color = np.mean([frame[point[1], point[0]] for point in box], axis=0)
    boundary_color = tuple(map(int, boundary_color))  # Convert to integer

    # Fill the bounding box with the boundary color
    cv2.fillConvexPoly(frame, box, boundary_color)

# Function to warp and overlay the transparent PES logo based on the orientation of the largest region
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Reshape the contour points for PCA
    contour_points = largest_contour.reshape(-1, 2).astype(np.float32)

    # Apply PCA to find the principal orientation of the contour
    centroid, principal_axis = get_pca_orientation(contour_points)

    # Smooth the PCA principal axis
    principal_axis = smooth_pca_orientation(principal_axis)

    # Fit a rotated bounding box based on the smoothed principal axis
    rect = cv2.minAreaRect(largest_contour)
    box = cv2.boxPoints(rect)  # Get the box points of the rotated rectangle
    box = np.int0(box)  # Convert to integer for drawing

    # Fill the bounding box with the boundary color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Define the source points (corners of the PES logo)
    src_pts = np.float32([[0, 0], [logo_w, 0], [logo_w, logo_h], [0, logo_h]])

    # Define the destination points (corners of the rotated bounding box)
    dst_pts = np.float32(box)

    # Compute the homography matrix
    homography_matrix, _ = cv2.findHomography(src_pts, dst_pts)

    # Warp the logo to fit the detected region using homography
    warped_logo = cv2.warpPerspective(logo, homography_matrix, (width, height))

    # Extract the alpha channel from the warped logo (if it exists)
    if warped_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = warped_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        warped_logo_rgb = warped_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((warped_logo.shape[0], warped_logo.shape[1]), dtype=np.float32)
        warped_logo_rgb = warped_logo

    # Create a mask for the logo region (where the warped logo should be applied)
    logo_mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillConvexPoly(logo_mask, box, 255)  # Mask for the logo region

    # Use the alpha channel to blend the warped logo onto the original frame
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[:, :, c] = (1 - logo_alpha) * frame[:, :, c] + logo_alpha * warped_logo_rgb[:, :, c]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


<ipython-input-57-82f08f40ec42>:98: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)  # Convert to integer for drawing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to fill the bounding box with the color of its boundary
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (average color of corners)
    boundary_color = np.mean([frame[point[1], point[0]] for point in box], axis=0)
    boundary_color = tuple(map(int, boundary_color))  # Convert to integer

    # Fill the bounding box with the boundary color
    cv2.fillConvexPoly(frame, box, boundary_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the boundary color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Create a mask for the logo region (where the resized logo should be applied)
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Downscale the PES logo to reduce quality
downscale_factor = 0.5  # Adjust this factor to control the logo quality
logo_image = cv2.resize(logo_image, (int(logo_image.shape[1] * downscale_factor), int(logo_image.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the image)
_, compressed_logo = cv2.imencode('.jpg', logo_image, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_image = cv2.imdecode(compressed_logo, cv2.IMREAD_UNCHANGED)

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to fill the bounding box with the color of its boundary
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (average color of corners)
    boundary_color = np.mean([frame[point[1], point[0]] for point in box], axis=0)
    boundary_color = tuple(map(int, boundary_color))  # Convert to integer

    # Fill the bounding box with the boundary color
    cv2.fillConvexPoly(frame, box, boundary_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the boundary color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.5  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to fill the bounding box with the color of its boundary
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (average color of corners)
    boundary_color = np.mean([frame[point[1], point[0]] for point in box], axis=0)
    boundary_color = tuple(map(int, boundary_color))  # Convert to integer

    # Fill the bounding box with the boundary color
    cv2.fillConvexPoly(frame, box, boundary_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the boundary color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.5  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize previous background color for smoothing
prev_background_color = None
smoothing_factor = 0.9  # Strong smoothing factor for slow color changes

# Function to apply strong smoothing to the background color
def smooth_background_color(new_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = new_color
    else:
        # Apply exponential smoothing to blend new color with previous one
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * new_color)
    return prev_background_color

# Function to fill the bounding box with the smoothed boundary color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (average color of corners)
    boundary_color = np.mean([frame[point[1], point[0]] for point in box], axis=0)
    boundary_color = tuple(map(int, boundary_color))  # Convert to integer

    # Smooth the boundary color
    smoothed_color = smooth_background_color(np.array(boundary_color))

    # Fill the bounding box with the smoothed boundary color
    cv2.fillConvexPoly(frame, box, tuple(map(int, smoothed_color)))

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the smoothed boundary color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.5  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize previous background color for smoothing
prev_background_color = None
smoothing_factor = 0.9  # Strong smoothing factor for slow color changes

# Function to apply strong smoothing to the background color
def smooth_background_color(new_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = new_color
    else:
        # Apply exponential smoothing to blend new color with previous one
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * new_color)
    return prev_background_color

# Function to fill the bounding box with the smoothed boundary color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (average color of corners)
    boundary_color = np.mean([frame[point[1], point[0]] for point in box], axis=0)
    boundary_color = tuple(map(int, boundary_color))  # Convert to integer

    # Smooth the boundary color
    smoothed_color = smooth_background_color(np.array(boundary_color))

    # Fill the bounding box with the smoothed boundary color
    cv2.fillConvexPoly(frame, box, tuple(map(int, smoothed_color)))

# Function to create a feathered mask for blending
def create_feathered_mask(box, width, height, feather_size=20):
    # Create a mask with a white rectangle in the bounding box area
    mask = np.zeros((height, width), dtype=np.float32)
    cv2.fillConvexPoly(mask, box, 1)

    # Ensure feather_size is odd
    feather_size = feather_size | 1  # Make sure feather_size is odd

    # Apply Gaussian blur to feather the edges of the mask
    feathered_mask = cv2.GaussianBlur(mask, (feather_size, feather_size), 0)

    return feathered_mask

# Function to warp and overlay the transparent PES logo with blending
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the smoothed boundary color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Create a feathered mask for blending the bounding box area with the surrounding frame
    feathered_mask = create_feathered_mask(box, width, height)

    # Crop the feathered mask to the bounding box region
    feathered_mask_cropped = feathered_mask[y:y+h, x:x+w]

    # Blend the bounding box area with the rest of the frame
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = feathered_mask_cropped * (logo_alpha * resized_logo_rgb[:, :, c] + (1 - logo_alpha) * frame[y:y+h, x:x+w, c]) + (1 - feathered_mask_cropped) * frame[y:y+h, x:x+w, c]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.5  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array

    # Apply K-means clustering to group similar colors
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)

    # Find the most common cluster (majority color)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]  # Index of the most frequent cluster

    # Get the centroid of the most frequent cluster (the representative color)
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]

    return tuple(map(int, majority_color))  # Return the majority color as a tuple of integers

# Function to fill the bounding box with the majority color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (all boundary pixels)
    boundary_pixels = [frame[point[1], point[0]] for point in box]

    # Compute the majority color using clustering
    majority_color = compute_majority_color(boundary_pixels)

    # Fill the bounding box with the majority color
    cv2.fillConvexPoly(frame, box, majority_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the majority color of the boundary before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(rescaled_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.25  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize smoothing variables
prev_background_color = None
prev_overlay = None
smoothing_factor = 0.5  # Strong smoothing factor for color
overlay_smoothing_factor = 0.7  # Smoothing factor for overlay transitions
color_threshold = 40  # Threshold to skip drastic color changes

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array

    # Apply K-means clustering to group similar colors
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)

    # Find the most common cluster (majority color)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]  # Index of the most frequent cluster

    # Get the centroid of the most frequent cluster (the representative color)
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]

    return tuple(map(int, majority_color))  # Return the majority color as a tuple of integers

# Function to apply smoothing to the background color and skip drastic changes
def smooth_background_color(current_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = np.array(current_color)
    else:
        # Calculate the color difference
        color_diff = np.linalg.norm(np.array(current_color) - prev_background_color)

        # Skip drastic changes
        if color_diff > color_threshold:
            return tuple(map(int, prev_background_color))

        # Exponential smoothing of the background color
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * np.array(current_color))

    # Return the smoothed color as a tuple of integers
    return tuple(map(int, prev_background_color))

# Function to fill the bounding box with the smoothed majority color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (all boundary pixels)
    boundary_pixels = [frame[point[1], point[0]] for point in box]

    # Compute the majority color using clustering
    majority_color = compute_majority_color(boundary_pixels)

    # Smooth the majority color across frames
    smoothed_color = smooth_background_color(majority_color)

    # Fill the bounding box with the smoothed color
    cv2.fillConvexPoly(frame, box, smoothed_color)

# Function to smooth the overlay between frames
def smooth_overlay(prev_overlay, new_overlay):
    if prev_overlay is None:
        return new_overlay
    else:
        # Apply overlay smoothing using exponential averaging
        smoothed_overlay = cv2.addWeighted(new_overlay, 1 - overlay_smoothing_factor, prev_overlay, overlay_smoothing_factor, 0)
        return smoothed_overlay

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    global prev_overlay
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return frame

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the smoothed majority color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Create an overlay for the logo
    new_overlay = np.zeros_like(frame)
    for c in range(3):  # Loop over the color channels (R, G, B)
        new_overlay[y:y+h, x:x+w, c] = logo_alpha * resized_logo_rgb[:, :, c]

    # Smooth the overlay between frames
    smoothed_overlay = smooth_overlay(prev_overlay, new_overlay)

    # Apply the smoothed overlay to the frame
    frame = cv2.addWeighted(frame, 1, smoothed_overlay, 1, 0)

    # Update the previous overlay
    prev_overlay = smoothed_overlay

    return frame

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    final_frame = warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(final_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.1  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize smoothing variables
prev_background_color = None
color_threshold = 40  # Threshold to skip drastic color changes
smoothing_factor = 0.5  # Smoothing factor for background color

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array

    # Apply K-means clustering to group similar colors
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)

    # Find the most common cluster (majority color)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]  # Index of the most frequent cluster

    # Get the centroid of the most frequent cluster (the representative color)
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]

    return tuple(map(int, majority_color))  # Return the majority color as a tuple of integers

# Function to apply smoothing to the background color and skip drastic changes
def smooth_background_color(current_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = np.array(current_color)
    else:
        # Calculate the color difference
        color_diff = np.linalg.norm(np.array(current_color) - prev_background_color)

        # Skip drastic changes
        if color_diff > color_threshold:
            return tuple(map(int, prev_background_color))

        # Exponential smoothing of the background color
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * np.array(current_color))

    # Return the smoothed color as a tuple of integers
    return tuple(map(int, prev_background_color))

# Function to fill the bounding box with the smoothed majority color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (all boundary pixels)
    boundary_pixels = [frame[point[1], point[0]] for point in box]

    # Compute the majority color using clustering
    majority_color = compute_majority_color(boundary_pixels)

    # Smooth the majority color across frames
    smoothed_color = smooth_background_color(majority_color)

    # Fill the bounding box with the smoothed color
    cv2.fillConvexPoly(frame, box, smoothed_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return frame

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the smoothed majority color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

    return frame

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    final_frame = warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(final_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.1  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize smoothing variables
prev_background_color = None
color_threshold = 40  # Threshold to skip drastic color changes
smoothing_factor = 0.5  # Smoothing factor for background color

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array

    # Apply K-means clustering to group similar colors
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)

    # Find the most common cluster (majority color)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]  # Index of the most frequent cluster

    # Get the centroid of the most frequent cluster (the representative color)
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]

    return tuple(map(int, majority_color))  # Return the majority color as a tuple of integers

# Function to apply smoothing to the background color and skip drastic changes
def smooth_background_color(current_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = np.array(current_color)
    else:
        # Calculate the color difference
        color_diff = np.linalg.norm(np.array(current_color) - prev_background_color)

        # Skip drastic changes
        if color_diff > color_threshold:
            return tuple(map(int, prev_background_color))

        # Exponential smoothing of the background color
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * np.array(current_color))

    # Return the smoothed color as a tuple of integers
    return tuple(map(int, prev_background_color))

# Function to fill the bounding box with the smoothed majority color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (all boundary pixels)
    boundary_pixels = [frame[point[1], point[0]] for point in box]

    # Compute the majority color using clustering
    majority_color = compute_majority_color(boundary_pixels)

    # Smooth the majority color across frames
    smoothed_color = smooth_background_color(majority_color)

    # Fill the bounding box with the smoothed color
    cv2.fillConvexPoly(frame, box, smoothed_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return frame

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the smoothed majority color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

    return frame

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    final_frame = warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Add CNN-like filtering (blurring and resolution downsampling)
    blurred_frame = cv2.GaussianBlur(final_frame, (3, 3), 0)

    # Apply a smaller downsampling for a smoother effect
    filtered_frame = cv2.resize(blurred_frame, (target_width // 2, target_height // 2))
    filtered_frame = cv2.resize(filtered_frame, (target_width, target_height))

    # Write the filtered frame to the output video
    out.write(filtered_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans

# Paths to your original and mask videos and the PES logo
# Paths to your original and mask videos and the PES logo
original_video_path = '/content/gyrocut.mp4'  # Original video (F1 footage)
mask_video_path = '/content/f1mask.mp4'  # Mask video showing sponsor spots
logo_path = '/content/deptlogo.png'  # PES University logo image (PNG with transparency)


# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.1  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_fps_adjustment.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize smoothing variables
prev_background_color = None
color_threshold = 40  # Threshold to skip drastic color changes
smoothing_factor = 0.5  # Smoothing factor for background color

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array

    # Apply K-means clustering to group similar colors
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)

    # Find the most common cluster (majority color)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]  # Index of the most frequent cluster

    # Get the centroid of the most frequent cluster (the representative color)
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]

    return tuple(map(int, majority_color))  # Return the majority color as a tuple of integers

# Function to apply smoothing to the background color and skip drastic changes
def smooth_background_color(current_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = np.array(current_color)
    else:
        # Calculate the color difference
        color_diff = np.linalg.norm(np.array(current_color) - prev_background_color)

        # Skip drastic changes
        if color_diff > color_threshold:
            return tuple(map(int, prev_background_color))

        # Exponential smoothing of the background color
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * np.array(current_color))

    # Return the smoothed color as a tuple of integers
    return tuple(map(int, prev_background_color))

# Function to fill the bounding box with the smoothed majority color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (all boundary pixels)
    boundary_pixels = [frame[point[1], point[0]] for point in box]

    # Compute the majority color using clustering
    majority_color = compute_majority_color(boundary_pixels)

    # Smooth the majority color across frames
    smoothed_color = smooth_background_color(majority_color)

    # Fill the bounding box with the smoothed color
    cv2.fillConvexPoly(frame, box, smoothed_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return frame

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the smoothed majority color before placing the logo
    fill_bounding_box(frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

    return frame

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (no PCA)
    final_frame = warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Add CNN-like filtering (blurring and aggressive resolution downsampling)
    blurred_frame = cv2.GaussianBlur(final_frame, (7, 7), 0)  # Larger kernel for stronger blur

    # Apply more aggressive downsampling
    filtered_frame = cv2.resize(blurred_frame, (target_width // 3, target_height // 3))  # Downsample more aggressively
    filtered_frame = cv2.resize(filtered_frame, (target_width, target_height))  # Rescale back to original size

    # Write the filtered frame to the output video
    out.write(filtered_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")


IndexError: index 720 is out of bounds for axis 0 with size 720

In [ ]:
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.1  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_pca_rotation.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Function to perform PCA and check if rotation by 90° is needed
def check_rotation_by_pca(contour):
    # Perform PCA on the contour points to determine the main orientation
    pca = PCA(n_components=1)
    contour_points = np.squeeze(contour)
    pca.fit(contour_points)

    # The first principal component gives the main orientation
    pc1 = pca.components_[0]

    # Calculate the angle between the main component and the horizontal axis
    angle = np.arctan2(pc1[1], pc1[0]) * 180 / np.pi  # Convert to degrees

    # If the angle is near vertical (close to 90° or -90°), we rotate the image
    if 75 <= abs(angle) <= 105:  # Check if the angle is approximately vertical
        return True
    return False

# Function to rotate the image by 90° if needed
def rotate_if_needed(frame, contour):
    if check_rotation_by_pca(contour):
        # Rotate the frame by 90 degrees clockwise
        rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        return rotated_frame, True
    return frame, False

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array

    # Apply K-means clustering to group similar colors
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)

    # Find the most common cluster (majority color)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]  # Index of the most frequent cluster

    # Get the centroid of the most frequent cluster (the representative color)
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]

    return tuple(map(int, majority_color))  # Return the majority color as a tuple of integers

# Function to fill the bounding box with the smoothed majority color
def fill_bounding_box(frame, box):
    # Sample the color from the boundary of the box (all boundary pixels)
    boundary_pixels = [frame[point[1], point[0]] for point in box]

    # Compute the majority color using clustering
    majority_color = compute_majority_color(boundary_pixels)

    # Fill the bounding box with the smoothed color
    cv2.fillConvexPoly(frame, box, majority_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return frame

    # Find the largest contour by area
    largest_contour = max(contours, key=cv2.contourArea)

    # Check if we need to rotate the frame based on PCA
    rotated_frame, rotated = rotate_if_needed(frame, largest_contour)

    # Update the contour points if the frame was rotated
    if rotated:
        # Need to rotate the contour coordinates as well if the frame was rotated
        largest_contour = np.array([[(p[0][1], target_height - p[0][0])] for p in largest_contour])

    # Fit a bounding box around the largest contour (no rotation, just a rectangle)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])

    # Fill the bounding box with the majority color
    fill_bounding_box(rotated_frame, box)

    # Get the size of the PES logo (assumes it has 4 channels, RGBA)
    logo_h, logo_w = logo.shape[:2]

    # Resize the logo to fit the bounding box
    resized_logo = cv2.resize(logo, (w, h))

    # Extract the alpha channel from the resized logo (if it exists)
    if resized_logo.shape[2] == 4:  # If the logo has an alpha channel
        logo_alpha = resized_logo[:, :, 3] / 255.0  # Normalize alpha to [0, 1]
        resized_logo_rgb = resized_logo[:, :, :3]  # Get the RGB part of the logo
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo

    # Apply the logo to the bounding box region
    for c in range(0, 3):  # Loop over the color channels (R, G, B)
        rotated_frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * rotated_frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]

    return rotated_frame

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    # Synchronize the mask frames with the original video
    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    # Rescale the original frame to 720p
    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))

    # Convert mask frame to grayscale (if it's not already)
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Threshold the mask to create a binary mask (adjust threshold if necessary)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Rescale the binary mask to match the rescaled frame size
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    # Warp and overlay the transparent PES logo based on the largest detected region (with PCA-based rotation)
    final_frame = warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Write the modified frame to the output video
    out.write(final_frame)

    # Increment the mask frame counter
    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import tensorflow as tf

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/neymarjrmadskillztekks.mp4'  # Original video
mask_video_path = '/content/sam2_masked_video_1726331540163.mp4'  # Mask video
logo_path = '/content/pes_logo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.1  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/output_rescaled_720p_with_cnn_filtering.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize smoothing variables
prev_background_color = None
color_threshold = 80  # Threshold to skip drastic color changes
smoothing_factor = 0.1  # Smoothing factor for background color

# Function to perform PCA and check if rotation by 90° is needed
def check_rotation_by_pca(contour):
    pca = PCA(n_components=1)
    contour_points = np.squeeze(contour)
    pca.fit(contour_points)
    pc1 = pca.components_[0]
    angle = np.arctan2(pc1[1], pc1[0]) * 180 / np.pi  # Convert to degrees
    if 75 <= abs(angle) <= 105:  # Check if the angle is approximately vertical
        return True
    return False

# Function to rotate the image by 90° if needed
def rotate_if_needed(frame, contour):
    if check_rotation_by_pca(contour):
        rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        return rotated_frame, True
    return frame, False

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]
    return tuple(map(int, majority_color))

# Function to apply smoothing to the background color and skip drastic changes
def smooth_background_color(current_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = np.array(current_color)
    else:
        color_diff = np.linalg.norm(np.array(current_color) - prev_background_color)
        if color_diff > color_threshold:
            return tuple(map(int, prev_background_color))
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * np.array(current_color))
    return tuple(map(int, prev_background_color))

# Function to fill the bounding box with the smoothed majority color
def fill_bounding_box(frame, box):
    boundary_pixels = [frame[point[1], point[0]] for point in box]
    majority_color = compute_majority_color(boundary_pixels)
    smoothed_color = smooth_background_color(majority_color)
    cv2.fillConvexPoly(frame, box, smoothed_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return frame
    largest_contour = max(contours, key=cv2.contourArea)
    rotated_frame, rotated = rotate_if_needed(frame, largest_contour)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])
    fill_bounding_box(rotated_frame, box)
    resized_logo = cv2.resize(logo, (w, h))
    if resized_logo.shape[2] == 4:
        logo_alpha = resized_logo[:, :, 3] / 255.0
        resized_logo_rgb = resized_logo[:, :, :3]
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo
    for c in range(0, 3):
        rotated_frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * rotated_frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]
    return rotated_frame

# CNN model for aggressive filtering
def cnn_filtering(image):
    image = np.expand_dims(image, axis=0)
    image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)
    low_res_image = tf.image.resize(image_tensor, [image_tensor.shape[1] // 2, image_tensor.shape[2] // 2])
    filtered_image = tf.image.resize(low_res_image, [image_tensor.shape[1], image_tensor.shape[2]])
    return filtered_image[0].numpy().astype(np.uint8)

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    final_frame = warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Apply Gaussian blur
    blurred_frame = cv2.GaussianBlur(final_frame, (5, 5), 0)

    # Apply CNN filtering
    cnn_filtered_frame = cnn_filtering(blurred_frame)

    out.write(cnn_filtered_frame)

    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import tensorflow as tf

# Paths to your original and mask videos and the PES logo
original_video_path = '/content/kimi_crop.mp4'  # Original video (F1 footage)
mask_video_path = '/content/smoothed_mask.mp4'  # Mask video showing sponsor spots
logo_path = '/content/deptlogo.png'  # PES University logo image (PNG with transparency)

# Load the PES University logo with transparency (RGBA)
logo_image = cv2.imread(logo_path, cv2.IMREAD_UNCHANGED)

# Check if the logo is loaded properly
if logo_image is None:
    raise FileNotFoundError(f"Could not load logo image from {logo_path}")

# Separate the RGB channels and the alpha channel
if logo_image.shape[2] == 4:  # Check if the image has an alpha channel
    logo_rgb = logo_image[:, :, :3]  # Extract RGB channels
    logo_alpha = logo_image[:, :, 3]  # Extract the alpha channel
else:
    logo_rgb = logo_image
    logo_alpha = np.ones((logo_image.shape[0], logo_image.shape[1]), dtype=np.uint8) * 255  # Default alpha channel

# Downscale the PES logo's RGB channels to reduce quality
downscale_factor = 0.1  # Adjust this factor to control the logo quality
logo_rgb = cv2.resize(logo_rgb, (int(logo_rgb.shape[1] * downscale_factor), int(logo_rgb.shape[0] * downscale_factor)))

# Apply JPEG compression to simulate lower quality (save and reload the RGB channels only)
_, compressed_rgb = cv2.imencode('.jpg', logo_rgb, [int(cv2.IMWRITE_JPEG_QUALITY), 60])
logo_rgb = cv2.imdecode(compressed_rgb, cv2.IMREAD_UNCHANGED)

# Resize the alpha channel to match the downscaled RGB channels
logo_alpha = cv2.resize(logo_alpha, (logo_rgb.shape[1], logo_rgb.shape[0]))

# Combine the downscaled RGB channels with the untouched alpha channel
logo_image = cv2.merge([logo_rgb, logo_alpha])

# Open both the original video and the mask video
original_cap = cv2.VideoCapture(original_video_path)
mask_cap = cv2.VideoCapture(mask_video_path)

# Get original video properties
original_width = int(original_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(original_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = original_cap.get(cv2.CAP_PROP_FPS)

mask_fps = mask_cap.get(cv2.CAP_PROP_FPS)

# Calculate the frame duplication ratio (24 FPS to 30 FPS means we duplicate roughly 1 out of every 5 frames)
frame_ratio = original_fps / mask_fps

# Target resolution (rescale to 720p)
target_height = 720
target_width = int((target_height / original_height) * original_width)  # Maintain aspect ratio

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/content/f1out.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), int(original_fps), (target_width, target_height))

# Initialize smoothing variables
prev_background_color = None
color_threshold = 40  # Threshold to skip drastic color changes
smoothing_factor = 0.5  # Smoothing factor for background color

# Function to perform PCA and check if rotation by 90° is needed
def check_rotation_by_pca(contour):
    pca = PCA(n_components=1)
    contour_points = np.squeeze(contour)
    pca.fit(contour_points)
    pc1 = pca.components_[0]
    angle = np.arctan2(pc1[1], pc1[0]) * 180 / np.pi  # Convert to degrees
    if 75 <= abs(angle) <= 105:  # Check if the angle is approximately vertical
        return True
    return False

# Function to rotate the image by 90° if needed
def rotate_if_needed(frame, contour):
    if check_rotation_by_pca(contour):
        rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        return rotated_frame, True
    return frame, False

# Function to compute the majority color by clustering similar colors
def compute_majority_color(boundary_pixels, num_clusters=2):
    boundary_pixels = np.array(boundary_pixels).reshape(-1, 3)  # Convert boundary pixels to 2D array
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(boundary_pixels)
    labels, counts = np.unique(kmeans.labels_, return_counts=True)
    dominant_cluster_index = labels[np.argmax(counts)]
    majority_color = kmeans.cluster_centers_[dominant_cluster_index]
    return tuple(map(int, majority_color))

# Function to apply smoothing to the background color and skip drastic changes
def smooth_background_color(current_color):
    global prev_background_color
    if prev_background_color is None:
        prev_background_color = np.array(current_color)
    else:
        color_diff = np.linalg.norm(np.array(current_color) - prev_background_color)
        if color_diff > color_threshold:
            return tuple(map(int, prev_background_color))
        prev_background_color = (smoothing_factor * prev_background_color) + ((1 - smoothing_factor) * np.array(current_color))
    return tuple(map(int, prev_background_color))

# Function to clamp bounding box coordinates to image dimensions
def clamp_to_bounds(point, width, height):
    x, y = point
    return (min(max(x, 0), width - 1), min(max(y, 0), height - 1))

# Function to fill the bounding box with the smoothed majority color
def fill_bounding_box(frame, box, width, height):
    # Ensure box points are within frame bounds
    box = [clamp_to_bounds(point, width, height) for point in box]

    # Sample the color from the boundary of the box (all boundary pixels)
    boundary_pixels = [frame[point[1], point[0]] for point in box]
    majority_color = compute_majority_color(boundary_pixels)
    smoothed_color = smooth_background_color(majority_color)
    cv2.fillConvexPoly(frame, np.array(box), smoothed_color)

# Function to warp and overlay the transparent PES logo based on the largest region detected
def warp_and_overlay_logo(frame, mask, width, height, logo):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return frame
    largest_contour = max(contours, key=cv2.contourArea)
    rotated_frame, rotated = rotate_if_needed(frame, largest_contour)
    x, y, w, h = cv2.boundingRect(largest_contour)
    box = np.array([[x, y], [x + w, y], [x + w, y + h], [x, y + h]])
    fill_bounding_box(rotated_frame, box, width, height)
    resized_logo = cv2.resize(logo, (w, h))
    if resized_logo.shape[2] == 4:
        logo_alpha = resized_logo[:, :, 3] / 255.0
        resized_logo_rgb = resized_logo[:, :, :3]
    else:
        logo_alpha = np.ones((resized_logo.shape[0], resized_logo.shape[1]), dtype=np.float32)
        resized_logo_rgb = resized_logo
    for c in range(0, 3):
        rotated_frame[y:y+h, x:x+w, c] = (1 - logo_alpha) * rotated_frame[y:y+h, x:x+w, c] + logo_alpha * resized_logo_rgb[:, :, c]
    return rotated_frame

# CNN model for aggressive filtering
def cnn_filtering(image):
    image = np.expand_dims(image, axis=0)
    image_tensor = tf.convert_to_tensor(image, dtype=tf.float32)
    low_res_image = tf.image.resize(image_tensor, [image_tensor.shape[1] // 2, image_tensor.shape[2] // 2])
    filtered_image = tf.image.resize(low_res_image, [image_tensor.shape[1], image_tensor.shape[2]])
    return filtered_image[0].numpy().astype(np.uint8)

# Variable to track which frame in the mask video to duplicate
mask_frame_counter = 0

# Process both videos frame by frame
while original_cap.isOpened() and mask_cap.isOpened():
    ret_orig, original_frame = original_cap.read()

    if mask_frame_counter % frame_ratio < 1:
        ret_mask, mask_frame = mask_cap.read()

    if not ret_orig or not ret_mask:
        break

    rescaled_frame = cv2.resize(original_frame, (target_width, target_height))
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)
    _, binary_mask = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)
    rescaled_mask = cv2.resize(binary_mask, (target_width, target_height))

    final_frame = warp_and_overlay_logo(rescaled_frame, rescaled_mask, target_width, target_height, logo_image)

    # Apply Gaussian blur
    blurred_frame = cv2.GaussianBlur(final_frame, (9, 9), 0)

    # Apply CNN filtering
    cnn_filtered_frame = cnn_filtering(blurred_frame)

    out.write(cnn_filtered_frame)

    mask_frame_counter += 1

# Release the video objects
original_cap.release()
mask_cap.release()
out.release()

# Download the output video in Colab
from google.colab import files
files.download(output_video_path)

print("Video processing complete.")

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` ex

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Video processing complete.


In [ ]:
import cv2

# Path to your mask video
mask_video_path = '/content/kimimask.mp4'  # Mask video (to smoothen shapes)

# Open the mask video
mask_cap = cv2.VideoCapture(mask_video_path)

# Get video properties
mask_fps = int(mask_cap.get(cv2.CAP_PROP_FPS))
frame_width = int(mask_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(mask_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create a VideoWriter object to save the smoothed output video
output_video_path = '/content/smoothed_mask.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), mask_fps, (frame_width, frame_height))

# Apply a large blur or smoothing technique to the mask
def smoothen_mask(mask_frame):
    # Apply a large Gaussian blur to smoothen the mask
    smoothed_mask = cv2.GaussianBlur(mask_frame, (9, 9), 0)  # Large kernel for strong smoothing

    # Optionally apply morphological operations to merge sections
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    morphed_mask = cv2.morphologyEx(smoothed_mask, cv2.MORPH_CLOSE, kernel)  # Morphological closing to merge sections

    return morphed_mask

# Process the mask video frame by frame
while mask_cap.isOpened():
    ret_mask, mask_frame = mask_cap.read()

    if not ret_mask:
        break

    # Convert mask frame to grayscale if it's in color
    mask_gray = cv2.cvtColor(mask_frame, cv2.COLOR_BGR2GRAY)

    # Apply smoothing and morphological transformations
    smoothed_mask = smoothen_mask(mask_gray)

    # Convert grayscale back to BGR format to save it as an MP4
    smoothed_mask_bgr = cv2.cvtColor(smoothed_mask, cv2.COLOR_GRAY2BGR)

    # Write the smoothed frame to the output video
    out.write(smoothed_mask_bgr)

# Release the video objects
mask_cap.release()
out.release()

# Download the smoothed mask video in Colab
from google.colab import files
files.download(output_video_path)

print("Mask smoothing complete.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mask smoothing complete.
